In [1]:
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
# ee.Initialize(project='gee-antonbiatov')
ee.Initialize(project='MY-GEE-PROJECT')

In [2]:
import subprocess
import random
import time

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    import geemap

In [3]:
#STARTYEAR = 2021
#ENDYEAR = 2021
STARTDAY=1
ENDDAY=250
INTERVAL = ENDDAY - STARTDAY
scale = 10

my_bands = ['B2', 'B3', 'B4', 'B8']

CLOUD_FILTER = 80



In [4]:
geom = ee.Geometry.Polygon(
        [
            [
             [50.230865478515625,26.309419586402797],[50.22674560546875,26.30934264658286],[50.2239990234375,26.309419586402797],[50.21721839904785,26.308650185904614],
             [50.21498680114746,26.308727126184305],[50.21309852600098,26.309419586402797],[50.17979621887207,26.3150360552725],[50.177650451660156,26.310881433275505],
             [50.175161361694336,26.3072652521349],[50.17250061035156,26.303418127049035],[50.16975402832031,26.301032845368663],[50.165462493896484,26.297878042601646],
             [50.1719856262207,26.292953300840438],[50.1774787902832,26.28972132539388],[50.18202781677246,26.288413119250144],[50.19155502319336,26.285565796050843],
             [50.19850730895996,26.28356493253137],[50.204429626464844,26.281794909113252],[50.20820617675781,26.280717490333167],[50.212240219116204,26.28002485869133],
             [50.22425651550293,26.27833174171643],[50.230865478515625,26.309419586402797]
            ]
        ]
)
feature = ee.Feature(geom, {})

AOI = feature.geometry()

In [5]:
def maskCloud1(image):
    qa = image.select('QA60');

  # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10;
    cirrusBitMask = 1 << 11;

  # Both flags should be set to zero, indicating clear conditions.
    mask = (qa.bitwiseAnd(cloudBitMask).eq(0)
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0)))

    return image.updateMask(mask)

def select_my_bands(img):
    return img.select(my_bands)

def maskCloud2(image):
  qa_1 = image.select('MSK_CLDPRB')
  mask = qa_1.lt(50)
  return image.updateMask(mask)

def get_s2_sr_col(aoi, STARTYEAR, ENDYEAR,STARTDAY,ENDDAY):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(AOI)
        .filter(ee.Filter.calendarRange(STARTYEAR, ENDYEAR, 'year'))
        .filter(ee.Filter.dayOfYear(STARTDAY, ENDDAY))
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
 #       .map(maskCloud1)
        .map(select_my_bands)
    )

    return ee.ImageCollection(s2_sr_col)



In [6]:
def median_image_generator(YEAR):
    s2_sr_col = get_s2_sr_col(AOI, YEAR, YEAR, STARTDAY,ENDDAY)
    med=s2_sr_col.median().clip(AOI).set('year',YEAR).set('col_size', s2_sr_col.size().getInfo())
    return med

In [7]:
YEAR = 2021

img = median_image_generator(YEAR)

print(img.getInfo())
print(img.get('col_size').getInfo())
print(img.get('year').getInfo())



/usr/local/lib/python3.12/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2_SR! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

  warnings.warn(warning, category=DeprecationWarning)


{'type': 'Image', 'bands': [{'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [2, 2], 'origin': [49, 25], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [2, 2], 'origin': [49, 25], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [2, 2], 'origin': [49, 25], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B8', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [2, 2], 'origin': [49, 25], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'col_size': 65, 'year': 2021, 'system:footprint': {'type': 'Polygon', 'coordinates': [[[50.230865478515625, 26.309419586402797], [50.22674560546875, 26.30934264658286], [50

In [8]:
YEARS = [2016, 2017, 2018, 2019, 2020, 2021]

In [9]:
#for YEAR in YEARS:
#    img = median_image_generator(YEAR)
#    print(str(YEAR)+' Image collection size: '+str(img.get('col_size').getInfo()))

In [10]:
rgbVis_False_color_482 = {'min': 500, 'max': 5500, 'bands': ['B4', 'B8', 'B2'],};

Map = []
center = AOI.centroid(10).coordinates().reverse().getInfo()
Map = geemap.Map(center=center, zoom=14)

Map.addLayer(AOI, {}, 'AOI', True, 1);

YEARS_to_export = []

for YEAR in YEARS:
    img = median_image_generator(YEAR)
    if img.get('col_size').getInfo() >= 1 :
        Map.addLayer(img, rgbVis_False_color_482, 'med_'+str(YEAR), True);
        print(str(YEAR)+' Image collection size: '+str(img.get('col_size').getInfo()))
        YEARS_to_export.append(YEAR)

print('YEARS_to_export: ', YEARS_to_export)

Map

2018 Image collection size: 2
2019 Image collection size: 65
2020 Image collection size: 75
2021 Image collection size: 65
YEARS_to_export:  [2018, 2019, 2020, 2021]


Map(center=[26.297459878673237, 50.20115646416002], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# Export Sentinel 2 data

for YEAR in YEARS_to_export:
    img = median_image_generator(YEAR)
    ee.batch.Export.image.toDrive(image = img, region=AOI, folder="GEE_data_export",
    description = 'Khobar_S2_SR_'+str(YEAR), crs='EPSG:3857', scale = scale, maxPixels = 1e10,fileFormat = 'GeoTIFF', formatOptions = { "cloudOptimized": True }).start()
    time.sleep(round(random.uniform(15, 35)))

